In [1]:
pip install tensorflow pillow numpy


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [1]:
pip install open3d tensorflow numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers

# Load and preprocess the image (for geometry compression)
def load_image(img_path, size=(128, 128)):
    img = Image.open(img_path)
    img = img.resize(size)  # Resize for geometry compression
    img = np.array(img) / 255.0  # Normalize to [0, 1] range
    return img

# Function to load the original texture
def load_texture_image(img_path, size=(128, 128)):
    img = Image.open(img_path)
    img = img.resize(size)  # Resize to match compressed geometry
    img = np.array(img) / 255.0  # Normalize the texture
    return img

# Build the CNN autoencoder for geometry compression
def build_geometry_compression_autoencoder():
    input_img = layers.Input(shape=(128, 128, 3))  # Input size of the image
    
    # Encoder: heavily reduce spatial dimensions to compress geometry
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    
    # Latent space: this part represents the compressed geometry
    encoded = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)

    # Decoder: attempt to recover geometry, but with compression effects
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    
    # Output the compressed geometry image
    decoded_geometry = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Define and compile the autoencoder model
    autoencoder = tf.keras.models.Model(input_img, decoded_geometry)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder

# Build and train the model for geometry compression
geometry_autoencoder = build_geometry_compression_autoencoder()
geometry_autoencoder.summary()

# Train the model with a single image (this step is optional if you load a pre-trained model)
# You can skip this step if you already trained and saved the model
# geometry_autoencoder.fit(geometry_image_data, geometry_image_data, epochs=50, batch_size=1)

# Alternatively, load the pre-trained model if you have already saved it
# geometry_autoencoder = tf.keras.models.load_model('geometry_compression_model.h5')

# Function to process each image in a folder
def process_images_in_folder(input_folder, output_folder, size=(128, 128)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_file in os.listdir(input_folder):
        if image_file.endswith('.png') or image_file.endswith('.jpg') or image_file.endswith('.jpeg'):
            input_image_path = os.path.join(input_folder, image_file)
            
            # Load the image and texture
            geometry_image_data = load_image(input_image_path, size)
            texture_image_data = load_texture_image(input_image_path, size)
            
            # Add batch dimension
            geometry_image_data = np.expand_dims(geometry_image_data, axis=0)
            texture_image_data = np.expand_dims(texture_image_data, axis=0)
            
            # Compress and decompress the geometry
            compressed_geometry = geometry_autoencoder.predict(geometry_image_data)
            
            # Apply the original texture back onto the compressed geometry
            final_output = np.clip(texture_image_data, 0, 1) * np.mean(compressed_geometry, axis=-1, keepdims=True)
            
            # Convert the result back to an image and save it, resizing back to 1024x1024
            final_output = np.squeeze(final_output)  # Remove batch dimension
            final_output = (final_output * 255).astype(np.uint8)  # Convert back to 0-255 range
            
            output_image = Image.fromarray(final_output).resize((1024, 1024))  # Resize back to original size
            output_image_path = os.path.join(output_folder, image_file)
            output_image.save(output_image_path)
            
            print(f"Processed and saved: {output_image_path}")

# Specify the input and output folders
input_folder = r'./source_folder'
output_folder = r'./destination_folder'

# Process all images in the input folder
process_images_in_folder(input_folder, output_folder)

print("All images processed successfully.")


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 16, 16, 256)    │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 32, 32, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_4 (UpSampling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_5 (UpSampling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 128, 128, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,101,443 (11.83 MB)

 Trainable params: 3,101,443 (11.83 MB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Processed and saved: F:/8iVFBv2/8iVFBv2/soldier/png_grey_compressed_images\soldier_vox10_0536.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
Processed and saved: F:/8iVFBv2/8iVFBv2/soldier/png_grey_compressed_images\soldier_vox10_0537.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
Processed and saved: F:/8iVFBv2/8iVFBv2/soldier/png_grey_compressed_images\soldier_vox10_0538.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 509ms/step
Processed and saved: F:/8iVFBv2/8iVFBv2/soldier/png_grey_compressed_images\soldier_vox10_0539.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
Processed and saved: F:/8iVFBv2/8iVFBv2/soldier/png_grey_compressed_images\soldier_vox10_0540.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
Processed and saved: F:/8iVFBv2/8iVFBv2/soldier/png_grey_compressed_images\soldier_vox10_0541.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step
Processed and saved: F:/8iVFBv2/8iVFBv2/soldier/png_grey_compressed_images\soldier_vox10_0542.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step
Proce